In [ ]:
# 설치 명령어
%pip install --upgrade pip
%pip install langchain langchain-openai dotenv

%pip list

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env")

In [ ]:

# openai_api_key = os.getenv("OPENAI_API_KEY")
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env")

print("Current Project:", os.environ["LANGSMITH_PROJECT"])


from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model="gpt-4o",
    temperature=1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
response = llm.invoke("한국의 수도는?")  # 이제 정상 작동해야 함
print(response.content)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

evaluation_prompt = ChatPromptTemplate.from_template(
    """다음 대화를 체크리스트 기준으로 평가하세요:
    - 적합성: {relevance_score}/5
    - 명확성: {clarity_score}/5
    - 완결성: {completeness_score}/5
    - 토큰 효율성: {efficiency_score}/5
    
    대화 내용: {conversation}
    총점: {total_score}/20
    개선 제안: {suggestions}"""
)
evaluation_chain = evaluation_prompt | ChatOpenAI()

def generate_feedback(response):
    if any(score < 3 for score in response['scores'].values()):
        return "다음과 같이 수정 제안: '답변을 더 구체적인 예시와 함께 재구성해보세요' :cite[6]"
    return "모든 기준 충족"

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationSummaryBufferMemory

# 1. 벡터 저장소 및 메모리 초기화
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())
memory = ConversationSummaryBufferMemory(llm=ChatOpenAI(), max_token_limit=500)

# 2. 평가 체인 통합
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0.5),
    retriever=vectorstore.as_retriever(),
    memory=memory,
    condense_question_prompt=CustomPromptTemplate(
        template="""현재 대화: {chat_history}
                    새로운 질문: {question}
                    → 평가 기준을 적용해 질문 재구성:"""
    )
)

# 3. 실행 및 자동 평가
response = qa_chain({"question": "기후 변화 대응 정책은?"})
evaluation_result = evaluation_chain.invoke({
    "conversation": response["answer"],
    "relevance_score": 4,
    "clarity_score": 3,
    "completeness_score": 2,
    "efficiency_score": 4
})
print(evaluation_result)

In [ ]:
import os
from langchain import PromptTemplate, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

##############################
# 환경 설정
##############################

# GPT-4 모델: 대화 생성용
gpt4_llm = ChatOpenAI(
    temperature=0.7,
    model_name="gpt-4",  # langchain에서 사용할 때 모델 이름 지정
    max_tokens=512
)

# GPT-3.5 모델: 첨삭(체크리스트)용
gpt35_llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",
    max_tokens=512
)

# 사용자 이름
u_name = "준희"

# 대화상대의 이름
op_name = "수빈"

##############################
# 1) 시나리오 + 상대 성격 입력
##############################
scenario = """당신({u_name})은 남자 주인공이며 20대 대학생입니다, 여성 주인공({op_name})과 수업에서는 가끔 마주치지만,
오늘 처음 학과 ot 술자리에서 만났습니다. 
둘 사이에는 미묘한 긴장감이 느껴지는데, 여성 주인공({op_name})은 술잔을 들고서 당신({u_name})에게 말을 걸어옵니다.
"""
character_personality = """
- 여성 주인공({op_name})은 20대 대학생이며 솔직하고 직설적이지만, 가끔 내면의 불안이 드러난다.
- 가벼운 농담과 감정 표현을 좋아한다.
"""

##############################
# 2) 시나리오 기반 '상대의 첫 답변' + '내가 답변하면 좋을 예시' 생성
##############################
template_for_first_response = """
당신은 시나리오 속 여성 주인공({op_name})입니다.

시나리오:
{scenario}

성격:
{personality}

위 정보를 참고해, 다음을 출력해주세요:
{op_name}: 여자주인공({op_name})이이 남자 주인공({u_name})에게 현재 상황에서 처음 건네는 말
{u_name}의 예시 답변: 남자 주인공({u_name})이 이렇게 답하면 좋겠다고 생각되는 간단한 예시 답변
"""

prompt_for_first_response = PromptTemplate(
    input_variables=["scenario", "personality"],
    template=template_for_first_response
)

first_response_chain = LLMChain(
    llm=gpt4_llm,
    prompt=prompt_for_first_response
)

initial_output = first_response_chain.run({
    "scenario": scenario,
    "personality": character_personality,
    "u_name": u_name,
    "op_name": op_name
})

print("[상대의 첫 대사 & 답변 예시]")
print(initial_output)

##############################
# 3) 나의 실제 답변을 입력하고 => 상대의 답변 생성
##############################
# 예시로 사용자 입력을 직접 변수로 세팅(실전에서는 input() 등을 통해 입력받을 수 있음)
user_answer = "아, 안녕하세요? 저는 술 안좋아해요 죄송합니다"

print("\n[나의 실제 답변: {}]".format(user_answer))

template_for_opponent_response = """
당신은 시나리오 속 여성 주인공({op_name})입니다.
지금까지의 대화:

- 남자 주인공({u_name}): {user_answer}

시나리오:
{scenario}
성격:
{personality}

여성 주인공({op_name})의 자연스러운 반응을 대사로 작성해주세요.
"""
prompt_for_opponent_response = PromptTemplate(
    input_variables=["user_answer", "scenario", "personality"],
    template=template_for_opponent_response
)

opponent_response_chain = LLMChain(
    llm=gpt4_llm,
    prompt=prompt_for_opponent_response
)

opponent_output = opponent_response_chain.run({
    "user_answer": user_answer,
    "scenario": scenario,
    "personality": character_personality,
    "profiles": profiles,
})

print("\n[상대의 답변]")
print(opponent_output)

##############################
# 4) 새로운 대화 흐름에 맞춰 '내가 답변하면 좋을 예시' 다시 생성
##############################
template_for_my_answer_suggestion = """
시나리오:
{scenario}
성격:
{personality}

지금까지 대화:
- 남자 주인공({u_name}) 대사: {user_answer}
- 여성 주인공({op_name}) 대사: {opponent_answer}

남자 주인공({u_name})이 이 상황에서 답변하면 좋을 만한 예시를 간단히 작성해주세요.
"""
prompt_for_my_answer_suggestion = PromptTemplate(
    input_variables=["scenario", "personality", "user_answer", "opponent_answer"],
    template=template_for_my_answer_suggestion
)

my_answer_suggestion_chain = LLMChain(
    llm=gpt4_llm,
    prompt=prompt_for_my_answer_suggestion
)

my_suggested_answer = my_answer_suggestion_chain.run({
    "scenario": scenario,
    "personality": character_personality,
    "user_answer": user_answer,
    "opponent_answer": opponent_output,
    "u_name": u_name,
    "op_name": op_name
})

print("\n하민의 예시 답변: ")
print(my_suggested_answer)

##############################
# 5) 체크리스트 기반 GPT-3.5 첨삭
##############################
checklist = """
1) 사용자의 감정이나 의도를 파악하려고 노력하는가?
    - 대화 내용에 알맞은 감정을 극적으로 표현한다
    - 대화 주제에 대한 청자의 입장을 고려한다
    - 상황과 분위기에 알맞게 대화 한다
2) 청자와 상호작용하며 대화를 이끄는가?
    - 대화 주제에 대한 청자의 입장을 고려한다
    - 청자의 반용과피드백을 실시간으로 수용한다
3) 상황과 분위기에 알맞게 대화 하는가?
    - 상황과 분위기에 알맞게 대화 한다
4) 대화의 목적과 내용을 효과적으로 구성하는가?
    - 대화의 의도와 목표를 정확하게 전달한다
    - 논점을 분명하게 요약한다
    - 청자가 흥미를 가질만한 주제와 내용으로 대화를 구성한다
5) 대화 중 예의를 지키고, 긍정적인 말투와 태도를 유지했는가?
    - 대화 내용에 알맞은 감정을 극적으로 표현한다
    - 대화 주제에 대한 청자의 입장을 고려한다
    - 화자가 강조하는 내용을 존중한다
    - 상황과 맥락을 이해하려 노력한다
    - 화자의 이야기를 평가하거나 판단하지 않는다
"""

template_for_refinement = """
아래는 사용자({u_name})의 답변 초안입니다:
"{user_answer}"

아래 체크리스트에 따라 이 답변을 냉정하게 평가해주세요.

체크리스트:
{checklist}

각 항목에 대한 첨삭 내용 숫자 기준으로 10점 만점으로 작성해주세요.
예시: 
1) 9점,
2) 8점, 
...
"""
prompt_for_refinement = PromptTemplate(
    input_variables=["user_answer", "checklist"],
    template=template_for_refinement
)

refinement_chain = LLMChain(
    llm=gpt35_llm,
    prompt=prompt_for_refinement
)

# 첨삭할 문장은 위에서 "내가 답변하면 좋을 예시"로 나온 my_suggested_answer라 가정
refined_answer = refinement_chain.run({
    "user_answer": my_suggested_answer,
    "checklist": checklist,
    "u_name": u_name,
    "op_name": op_name
})

print("\n[체크리스트에 따른 첨삭 결과 (GPT-3.5)]")
print(refined_answer)


In [ ]:
import os
from langchain import PromptTemplate, LLMChain
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

#########################
# 0. 환경설정
#########################

# GPT-4 모델 (상대 답변 및 예시 답변 생성)
gpt4_llm = ChatOpenAI(
    temperature=0.7,
    model_name="gpt-4o-mini",
    max_tokens=512
)

# GPT-3.5 모델 (체크리스트 기반 첨삭)
gpt35_llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o-mini",
    max_tokens=512
)

#########################
# 시나리오 세팅
#########################
profiles = {
    "u_name": "하민",  # 남자 주인공(유저)
    "u_gender": "남자",  # 남자 주인공 성별
    "u_age": "21",  # 남자 주인공 나이
    "op_name": "은수",  # 여자 주인공(상대)
    "op_gender": "여자",  # 여자 주인공 성별
    "op_age": "21"  # 여자 주인공 나이
}

scenario = """당신({u_name})은 20대 대학생이며,
상대({op_name})을 같은 수업에서 가끔 봤지만
오늘 처음으로 학과 OT 술자리에서 제대로 대화를 나누게 되었습니다.
상대({op_name})는 술잔을 들고 당신({u_name})에게 말을 걸어옵니다.
사실 상대({op_name})는 당신({u_name})에게 관심은 없지만 
다들 친해지려고 노력하는 분위기에 단순히 맞추는 것 뿐입니다.
전혀 당신({u_name})에게 관심이 없는 상태입니다.
""".format(**profiles)

character_personality = """
상대({op_name})의 정보:
- {op_age}세,{op_gender} 한국인입니다.
- 솔직하고 직설적입니다.
- 시니컬한 성격입니다.
- INTP 성격 유형입니다.
- 말이 안통하면 짜증내는 성격입니다.
- 관심이 없는 상대에게는 무관심한 태도와 짧은 대답만 합니다.
- 부연설명을 하지 않습니다.

당신({u_name})의 정보:
- {u_age}세, {u_gender} 한국인입니다.
- 소심하지만, 대화를 통해 친해지고 싶어합니다.
- 노력하는 성격입니다.
""".format(**profiles)

#########################
# 시나리오에 맞춰 상대의 답변,내 답변 예시 생성
#########################
init_prompt = """
당신은 시나리오 속 대화 상대인({op_name})입니다.
시나리오상황에 집중하고, 성격을 잘 살려서 다음을 작성하라.

시나리오:
{scenario}

성격:
{personality}

당신이 출력 해야하는 것:
1. {op_name}의 첫 대사
2. {u_name}의 예시 답변

출력 형식:
{op_name}의 대사와 {u_name}의 예시 답변을 ','로 구분하여 작성
"""

# 프롬프트 템플릿에 변수를 전달하기 위해 PromptTemplate 사용
init_prompt_template = PromptTemplate(
    input_variables=["scenario", "personality", "op_name", "u_name"],
    template=init_prompt
)


init_chain = LLMChain(
    llm=gpt4_llm,
    prompt=init_prompt_template
)

init_output = init_chain.run({
    "scenario": scenario,
    "personality": character_personality,
    **profiles
})


#########################
print("[초기 출력]")
init_output_list = init_output.split(',')
print("[상대의 첫 대사]")
print(init_output_list[0])
print("\n[내가 답변하면 좋을 예시]")
print(init_output_list[1])

# 초기 출력 끝
# 대화 시작
#########################


# 대화 기록 (문자열 형태로 관리)
conversation_history = f"[시나리오]\n{scenario}\n\n[성격]\n{character_personality}\n\n[대화내용]\n{init_output_list[0]}\n"

#########################
# 2. 반복 입력으로 대화 진행
#########################
opponent_response = """
당신은 시나리오 속 인물({op_name})이다.
시나리오상황에 집중하고, 성격을 잘 살려야한다. 
이 상황에서 당신({op_name})이 답변한다면?

지금까지의 대화:
{conversation_history}

출력 형식:
{op_name}: "~~~"
"""

opponent_response_template = PromptTemplate(
    input_variables=["conversation_history", "user_answer", "op_name", "u_name"],
    template=opponent_response
)

opponent_chain = LLMChain(
    llm=gpt4_llm,
    prompt=opponent_response_template
)

# 다음 내 답변 예시 생성용 프롬프트
my_answer_suggest_prompt = """
지금까지의 대화:
{conversation_history}

체크리스트:
{checklist}

이 상황에서 상대({op_name})의 마지막 말에 대한 당신({u_name})의 답변을 체크리스트를 참조하고 대화 흐름에 맞게 작성해주세요.
"""

my_answer_suggest_template = PromptTemplate(
    input_variables=["conversation_history", "opponent_answer", "u_name", "op_name", "checklist"],
    template=my_answer_suggest_prompt
)

my_answer_suggest_chain = LLMChain(
    llm=gpt4_llm,
    prompt=my_answer_suggest_template
)

# 체크리스트 기반 첨삭
checklist = """
1) 사용자의 감정이나 의도를 파악하려고 노력하는가?
    - 대화 내용에 알맞은 감정을 극적으로 표현한다
    - 대화 주제에 대한 청자의 입장을 고려한다
    - 상황과 분위기에 알맞게 대화 한다
2) 청자와 상호작용하며 대화를 이끄는가?
    - 대화 주제에 대한 청자의 입장을 고려한다
    - 청자의 반용과피드백을 실시간으로 수용한다
3) 상황과 분위기에 알맞게 대화 하는가?
    - 상황과 분위기에 알맞게 대화 한다
4) 대화의 목적과 내용을 효과적으로 구성하는가?
    - 대화의 의도와 목표를 정확하게 전달한다
    - 논점을 분명하게 요약한다
    - 청자가 흥미를 가질만한 주제와 내용으로 대화를 구성한다
5) 대화 중 예의를 지키고, 긍정적인 말투와 태도를 유지했는가?
    - 대화 내용에 알맞은 감정을 극적으로 표현한다
    - 대화 주제에 대한 청자의 입장을 고려한다
    - 화자가 강조하는 내용을 존중한다
    - 상황과 맥락을 이해하려 노력한다
    - 화자의 이야기를 평가하거나 판단하지 않는다
"""

refine_prompt = """
당신은 커뮤니케이션 전문가 입니다.
당신은 남자 주인공({u_name})에게 도움이되는 냉정한 평가를 해야한다.
지금까지의 전체 대화 내용:
{conversation_history}

[체크리스트]
{checklist}

위 체크리스트와 대화 흐름을 바탕으로,
- 남자 주인공({u_name})의 마지막 답변이 자연스러운지 100점 만점으로 각 항목별 점수를 매기고
- 체크리스트의 모든 항목을 총합한 평가를 마지막에 짧게 20자 이내로 작성해주세요.
- 냉정한 평가가 중요합니다.
- 애매한 점수 말고 명확한 점수를 매겨야 한다.
- 100 점인 답변 : {suggested_answer}

출력 예시:
1) 80점
2) 15점
...
[총평] " ~~~~ "
"""

refine_template = PromptTemplate(
    input_variables=["conversation_history", "user_answer", "checklist", "u_name"],
    template=refine_prompt
)

refinement_chain = LLMChain(
    llm=gpt35_llm,
    prompt=refine_template
)

#########################
# 3. 메인 루프
#########################
print("\n[대화를 시작합니다. 'quit'을 입력하면 종료합니다.]")

u_name = profiles["u_name"]
op_name = profiles["op_name"]

while True:
    # (B) 내 답변: 사용자에게 입력
    user_input = input(f"\n{u_name}의 답변(종료시 'quit'): ")
    if user_input.lower().strip() == "quit":
        print("대화를 종료합니다.")
        break
    
    print(f"\n[{u_name}의 답변]")
    print(user_input)

    # 대화 기록에 추가
    conversation_history += f"\n[{u_name}]\n{user_input}\n"

    # (E) 체크리스트 기반 첨삭
    refine_result = refinement_chain.run({
        "conversation_history": conversation_history,
        "user_input": user_input,
        "checklist": checklist,
        "u_name": u_name,
        "suggested_answer": suggested_answer
    })

    # (C) 상대의 답변 생성 (GPT-4)
    opponent_answer = opponent_chain.run({
        "op_name": op_name,
        "u_name": u_name,
        "user_answer": user_input,
        "conversation_history": conversation_history
    })
    print(f"\n[상대({op_name})의 답변]")
    print(opponent_answer)
    
    # 대화 기록에 추가
    conversation_history += f"\n[{op_name}]\n{opponent_answer}\n"

    # (D) 다음 내 답변 예시 생성 (GPT-4)
    suggested_answer = my_answer_suggest_chain.run({
        "conversation_history": conversation_history,
        "opponent_answer": opponent_answer,
        "u_name": u_name,
        "op_name": op_name,
        "checklist": checklist
    })

    print(f"\n[{u_name}가 이렇게 답해볼 수 있음(예시)]")
    print(suggested_answer)


    print("\n[체크리스트 평가 & 수정 제안]")
    print(refine_result)

    # loop 반복 - 다시 B[내 답변]로

print("\n--- 대화가 끝났습니다. 감사합니다. ---")


In [1]:
import os
from langchain import PromptTemplate, LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

#########################
# 0. 환경설정
#########################

# GPT-4 모델
gpt4_llm = ChatOpenAI(
    temperature=0.7,
    model_name="gpt-4o-mini",
    max_tokens=512
)

#########################
# 시나리오 세팅
#########################
profiles = {
    "u_name": "하민",  # 남자 주인공(유저)
    "u_gender": "남자",  # 남자 주인공 성별
    "u_age": "21",  # 남자 주인공 나이
    "op_name": "은수",  # 여자 주인공(상대)
    "op_gender": "여자",  # 여자 주인공 성별
    "op_age": "21"  # 여자 주인공 나이
}

scenario = """당신({u_name})은 20대 대학생이며,
상대({op_name})을 같은 수업에서 가끔 봤지만
오늘 처음으로 학과 OT 술자리에서 제대로 대화를 나누게 되었습니다.
상대({op_name})는 술잔을 들고 당신({u_name})에게 말을 걸어옵니다.
사실 상대({op_name})는 당신({u_name})에게 관심은 없지만 
다들 친해지려고 노력하는 분위기에 단순히 맞추는 것 뿐입니다.
전혀 당신({u_name})에게 관심이 없는 상태입니다.
""".format(**profiles)

character_personality = """
상대({op_name})의 정보:
- {op_age}세,{op_gender} 한국인입니다.
- 솔직하고 직설적입니다.
- 시니컬한 성격입니다.
- INTP 성격 유형입니다.
- 말이 안통하면 짜증내는 성격입니다.
- 관심이 없는 상대에게는 무관심한 태도와 짧은 대답만 합니다.
- 부연설명을 하지 않습니다.

당신({u_name})의 정보:
- {u_age}세, {u_gender} 한국인입니다.
- 소심하지만, 대화를 통해 친해지고 싶어합니다.
- 노력하는 성격입니다.
""".format(**profiles)

#########################
# init_scenario
#########################
init_scenario = """
{op_name}: "안녕하세요. 오늘 처음으로 제대로 대화를 나누네요. {u_name}씨 맞죠?"
""".format(**profiles)


# 대화 기록 (문자열 형태로 관리)
conversation_history = f"[시나리오]\n{scenario}\n\n[성격]\n{character_personality}\n\n[대화내용]\n{init_scenario}\n"

#########################
# 2. 반복 입력으로 대화 진행
#########################
opponent_response = """
당신은 시나리오 속 인물({op_name})이다.
시나리오상황에 집중하고, 성격을 잘 살려야한다. 
이 상황에서 당신({op_name})이 답변한다면?

지금까지의 대화:
{conversation_history}

출력 형식:
{op_name}: "~~~"
"""

opponent_response_template = PromptTemplate(
    input_variables=["conversation_history", "user_answer", "op_name", "u_name"],
    template=opponent_response
)

# opponent_chain = LLMChain(
#     llm=gpt4_llm,
#     prompt=opponent_response_template
# )

opponent_chain = opponent_response_template | gpt4_llm | StrOutputParser()

# 다음 내 답변 예시 생성용 프롬프트
my_answer_suggest_prompt = """
지금까지의 대화:
{conversation_history}

체크리스트:
{checklist}

이 상황에서 상대({op_name})의 마지막 말에 대한 당신({u_name})의 답변을 체크리스트를 참조하고 대화 흐름에 맞게 작성해주세요.
"""

my_answer_suggest_template = PromptTemplate(
    input_variables=["conversation_history", "u_name", "op_name", "checklist"],
    template=my_answer_suggest_prompt
)

# my_answer_suggest_chain = LLMChain(
#     llm=gpt4_llm,
#     prompt=my_answer_suggest_template
# )

my_answer_suggest_chain = my_answer_suggest_template | gpt4_llm | StrOutputParser()

# 체크리스트 기반 첨삭
checklist = """
1) 사용자의 감정이나 의도를 파악하려고 노력하는가?
    - 대화 내용에 알맞은 감정을 극적으로 표현한다
    - 대화 주제에 대한 청자의 입장을 고려한다
    - 상황과 분위기에 알맞게 대화 한다
2) 청자와 상호작용하며 대화를 이끄는가?
    - 대화 주제에 대한 청자의 입장을 고려한다
    - 청자의 반용과피드백을 실시간으로 수용한다
3) 상황과 분위기에 알맞게 대화 하는가?
    - 상황과 분위기에 알맞게 대화 한다
4) 대화의 목적과 내용을 효과적으로 구성하는가?
    - 대화의 의도와 목표를 정확하게 전달한다
    - 논점을 분명하게 요약한다
    - 청자가 흥미를 가질만한 주제와 내용으로 대화를 구성한다
5) 대화 중 예의를 지키고, 긍정적인 말투와 태도를 유지했는가?
    - 대화 내용에 알맞은 감정을 극적으로 표현한다
    - 대화 주제에 대한 청자의 입장을 고려한다
    - 화자가 강조하는 내용을 존중한다
    - 상황과 맥락을 이해하려 노력한다
    - 화자의 이야기를 평가하거나 판단하지 않는다
"""

refine_prompt = """
당신은 커뮤니케이션 전문가 입니다.
당신은 남자 주인공({u_name})에게 도움이되는 냉정한 평가를 해야한다.
지금까지의 전체 대화 내용:
{conversation_history}

[체크리스트]
{checklist}

위 체크리스트와 대화 흐름을 바탕으로,
- 남자 주인공({u_name})의 마지막 답변이 자연스러운지 100점 만점으로 각 항목별 점수를 매기고
- 체크리스트의 모든 항목을 총합한 평가를 마지막에 짧게 20자 이내로 작성해주세요.
- 냉정한 평가가 중요합니다.
- 애매한 점수 말고 명확한 점수를 매겨야 한다.

출력 예시:
1) 80점
2) 15점
...
[총평] " ~~~~ "
"""

refine_template = PromptTemplate(
    input_variables=["conversation_history", "user_answer", "checklist", "u_name"],
    template=refine_prompt
)

# refinement_chain = LLMChain(
#     llm=gpt4_llm,
#     prompt=refine_template
# )

refinement_chain = refine_template | gpt4_llm | StrOutputParser()

#########################
# 3. 메인 루프
#########################
print("\n[대화를 시작합니다. 'quit'을 입력하면 종료합니다.]")

u_name = profiles["u_name"]
op_name = profiles["op_name"]

print(conversation_history)

# 대화 힌트
suggested_answer = my_answer_suggest_chain.invoke({
    "conversation_history": conversation_history,
    "u_name": u_name,
    "op_name": op_name,
    "checklist": checklist
})

print(f"\n[대화 힌트]")
print(suggested_answer)

while True:
    # (B) 내 답변: 사용자에게 입력
    user_input = input(f"\n{u_name}의 답변(종료시 'quit'): ")
    if user_input.lower().strip() == "quit":
        print("대화를 종료합니다.")
        break
    
    print(f"\n[{u_name}의 답변]")
    print(user_input)

    # 대화 기록에 추가
    conversation_history += f"\n[{u_name}]\n{user_input}\n"

    # (E) 체크리스트 기반 첨삭
    refine_result = refinement_chain.invoke({
        "conversation_history": conversation_history,
        "user_input": user_input,
        "checklist": checklist,
        "u_name": u_name,
        "suggested_answer": suggested_answer
    })

    print("\n[체크리스트 평가 & 수정 제안]")
    print(refine_result)

    # (C) 상대의 답변 생성 (GPT-4)
    opponent_answer = opponent_chain.invoke({
        "op_name": op_name,
        "u_name": u_name,
        "user_answer": user_input,
        "conversation_history": conversation_history
    })
    print(f"\n[상대({op_name})의 답변]")
    print(opponent_answer)
    
    # 대화 기록에 추가
    conversation_history += f"\n[{op_name}]\n{opponent_answer}\n"

    # (D) 다음 내 답변 예시 생성 (GPT-4)
    suggested_answer = my_answer_suggest_chain.invoke({
        "conversation_history": conversation_history,
        "opponent_answer": opponent_answer,
        "u_name": u_name,
        "op_name": op_name,
        "checklist": checklist
    })

    print(f"\n[대화 힌트]")
    print(suggested_answer)

    # loop 반복 - 다시 B[내 답변]로

print("\n--- 대화가 끝났습니다. 감사합니다. ---")



[대화를 시작합니다. 'quit'을 입력하면 종료합니다.]
[시나리오]
당신(하민)은 20대 대학생이며,
상대(은수)을 같은 수업에서 가끔 봤지만
오늘 처음으로 학과 OT 술자리에서 제대로 대화를 나누게 되었습니다.
상대(은수)는 술잔을 들고 당신(하민)에게 말을 걸어옵니다.
사실 상대(은수)는 당신(하민)에게 관심은 없지만 
다들 친해지려고 노력하는 분위기에 단순히 맞추는 것 뿐입니다.
전혀 당신(하민)에게 관심이 없는 상태입니다.


[성격]

상대(은수)의 정보:
- 21세,여자 한국인입니다.
- 솔직하고 직설적입니다.
- 시니컬한 성격입니다.
- INTP 성격 유형입니다.
- 말이 안통하면 짜증내는 성격입니다.
- 관심이 없는 상대에게는 무관심한 태도와 짧은 대답만 합니다.
- 부연설명을 하지 않습니다.

당신(하민)의 정보:
- 21세, 남자 한국인입니다.
- 소심하지만, 대화를 통해 친해지고 싶어합니다.
- 노력하는 성격입니다.


[대화내용]

은수: "안녕하세요. 오늘 처음으로 제대로 대화를 나누네요. 하민씨 맞죠?"



[대화 힌트]
하민: "네, 맞아요! 오늘 이렇게 이야기하게 돼서 반가워요. 은수씨는 학과 생활 어때요? 적응 잘하고 있어요?"
대화를 종료합니다.

--- 대화가 끝났습니다. 감사합니다. ---


In [11]:
import json
import asyncio
from textwrap import dedent
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from typing import Optional, Dict
from pydantic import ValidationError
from typing_extensions import Annotated, TypedDict
from langchain_core.output_parsers.json import JsonOutputParser
from langchain.chains import LLMChain

load_dotenv(dotenv_path=".env")

#########################
# 환경설정
#########################
gpt4_llm = ChatOpenAI(
    temperature=0.7,
    model_name="gpt-4o-mini",
    max_tokens=512
)

gpt4_llm_check = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini",
    max_tokens=512
)

#########################
# 시나리오 세팅
#########################
profiles = {
    "u_name": "하민",
    "u_gender": "남자",
    "u_age": "21",
    "op_name": "민희",
    "op_gender": "여자",
    "op_age": "21"
}

scenario = dedent("""\
    당신({u_name})은 20대 대학생이며,
    상대({op_name})을 같은 수업에서 가끔 봤지만
    오늘 처음으로 학과 OT 술자리에서 제대로 대화를 나누게 되었습니다.
    상대({op_name})는 술잔을 들고 당신({u_name})에게 말을 걸어옵니다.
    사실 상대({op_name})는 당신({u_name})에게 관심은 없지만 
    다들 친해지려고 노력하는 분위기에 단순히 맞추는 것 뿐입니다.
    전혀 당신({u_name})에게 관심이 없는 상태입니다.
""").format(**profiles)

character_personality = dedent("""\
    상대({op_name})의 정보:
    - {op_age}세,{op_gender} 한국인입니다.
    - 솔직하고 직설적입니다.
    - 시니컬한 성격입니다.
    - INTP 성격 유형입니다.
    - 말이 안통하면 짜증내는 성격입니다.
    - 관심이 없는 상대에게는 무관심한 태도와 짧은 대답만 합니다.
    - 부연설명을 하지 않습니다.

    당신({u_name})의 정보:
    - {u_age}세, {u_gender} 한국인입니다.
    - 소심하지만, 대화를 통해 친해지고 싶어합니다.
    - 노력하는 성격입니다.
""").format(**profiles)

# 첫 번째 성격
character_personality_1 = dedent("""\
    상대({op_name})의 정보:
    - {op_age}세, {op_gender} 한국인입니다.
    - 온화하고 상냥한 편입니다.
    - 타인의 감정을 잘 살피고 배려합니다.
    - ISFJ 성격 유형입니다.
    - 갈등을 피하려 하고 조용히 의견을 전달합니다.
    - 주변을 돌보는 것에 만족을 느끼는 편입니다.
    - 자기 이야기는 길게 하지 않습니다.
                                 
    당신({u_name})의 정보:
    - {u_age}세, {u_gender} 한국인입니다.
    - 소심하지만, 대화를 통해 친해지고 싶어합니다.
    - 노력하는 성격입니다.
""").format(**profiles)

# 두 번째 성격
character_personality_2 = dedent("""\
    상대({op_name})의 정보:
    - {op_age}세, {op_gender} 한국인입니다.
    - 에너지가 넘치고 외향적입니다.
    - 새로운 경험을 좋아하고 도전적입니다.
    - ENFP 성격 유형입니다.
    - 주변 사람들을 즐겁게 만드는 분위기 메이커입니다.
    - 말이 빠르고 풍부한 표현을 사용합니다.
    - 관심이 떨어지면 쉽게 다른 주제로 넘어갑니다.
                            
    당신({u_name})의 정보:
    - {u_age}세, {u_gender} 한국인입니다.
    - 소심하지만, 대화를 통해 친해지고 싶어합니다.
    - 노력하는 성격입니다.                               
""").format(**profiles)

# 세 번째 성격
character_personality_3 = dedent("""\
    상대({op_name})의 정보:
    - {op_age}세, {op_gender} 한국인입니다.
    - 차분하며 분석적인 성향을 보입니다.
    - 논리적 근거가 부족하면 선뜻 납득하지 않습니다.
    - ISTJ 성격 유형입니다.
    - 규칙과 원칙을 중시하며 시간을 엄수합니다.
    - 불확실성보다는 명확한 목표를 추구합니다.
    - 필요 이상의 말을 하지 않는 편입니다.
                                 
    당신({u_name})의 정보:
    - {u_age}세, {u_gender} 한국인입니다.
    - 소심하지만, 대화를 통해 친해지고 싶어합니다.
    - 노력하는 성격입니다.
""").format(**profiles)

init_scenario = dedent("""\
    [시나리오 시작]
    {op_name}: "안녕하세요. 오늘 처음으로 제대로 대화를 나누네요. {u_name}씨 맞죠?"
""").format(**profiles)

conversation_history = f"{scenario}\n\n{character_personality}\n\n{init_scenario}"

#########################
# 체인 설정
#########################
checklist = dedent("""\
【채점 진행 전 주의사항 및 평가 기준】  
1. **일관된 평가 기준 사용**  
   - 모든 평가자는 아래 제시된 세부 기준을 참고하여 채점합니다.  
   - 예시:  
     - **0~24점:** 매우 부족 - 항목의 요구사항을 거의 충족하지 못함.  
     - **25~49점:** 부족 - 요구사항의 일부만 충족하며, 전반적인 효과가 미흡함.  
     - **50~74점:** 보통 - 기본 요구사항은 충족하지만, 개선의 여지가 있음.  
     - **75~89점:** 우수 - 대부분의 기준을 잘 충족하며, 대화의 질이 높음.  
     - **90~100점:** 매우 우수 - 모든 기준을 완벽하게 충족하며, 뛰어난 대화 진행.

2. **세부 평가 항목 및 예시**  
   - 각 항목별로 아래의 세부 기준을 참고하여 평가하세요.

【체크리스트 및 세부 평가 기준】  
1) **사용자의 감정이나 의도를 파악하려고 노력하는가?**  
   - **세부 기준:**  
     a. 대화 내용에 맞는 감정을 극적으로 표현하는가?  
     b. 대화 주제에 대한 청자의 입장을 충분히 고려하는가?  
     c. 상황과 분위기에 맞는 대화 방식을 사용하는가?  
   - **평가 예시:**  
     - 0~24점: 사용자의 감정이나 의도 파악 시도조차 보이지 않으며, 감정 표현이 전혀 없거나 오히려 부적절하게 왜곡됨.
     - 25~49점: 감정 표현이나 의도 파악의 시도가 있으나, 매우 제한적이며 청자의 입장이나 상황을 거의 고려하지 않음.
     - 50~74점: 기본적인 감정 표현과 의도 파악은 있으나, 미묘한 감정의 변화나 상황에 따른 세심한 표현이 부족함.
     - 75~89점: 대화 전반에서 감정과 의도의 파악이 잘 이루어지며, 대부분 상황과 청자의 입장을 반영하지만, 일부 세부 표현에서 개선의 여지가 있음.
     - 90~100점: 사용자의 감정과 의도를 정확히 파악하여, 상황에 맞는 극적인 감정 표현과 청자의 입장을 완벽하게 반영한 답변을 제공함.

2) **청자와 상호작용하며 대화를 이끄는가?**  
   - **세부 기준:**  
     a. 대화 주제에 대한 청자의 입장을 지속적으로 고려하는가?  
     b. 청자의 반응과 피드백을 실시간으로 수용하여 대화에 반영하는가?  
   - **평가 예시:**  
     - 0~24점: 청자와의 상호작용이 전혀 이루어지지 않으며, 청자의 피드백이나 반응에 대해 무관심하게 대화를 진행함.
     - 25~49점: 청자의 반응을 일부 인지하지만, 거의 반영하지 못하고 일방적으로 대화를 주도하는 모습이 두드러짐.
     - 50~74점: 청자의 일부 피드백은 반영하나, 대화의 흐름이 다소 어색하고 일관성이 떨어지며, 상호작용이 제한적임.
     - 75~89점: 청자의 피드백과 반응을 적절히 수용하여 대화를 원활하게 이끌지만, 약간의 미세한 조정이 필요한 경우가 있음.
     - 90~100점: 청자의 피드백에 즉각적이고 효과적으로 반응하며, 자연스럽게 대화의 흐름을 이끌어가면서 청자와 완벽한 상호작용을 보여줌.

3) **상황과 분위기에 알맞게 대화 하는가?**  
   - **세부 기준:**  
     a. 대화가 진행되는 상황 및 분위기를 충분히 고려하여 적절한 어휘와 표현을 사용하는가?  
   - **평가 예시:**  
     - 0~24점: 대화의 상황이나 분위기를 전혀 고려하지 않고, 부적절하거나 혼란스러운 어휘와 표현을 사용함.
     - 25~49점: 상황이나 분위기에 대한 반영이 극히 미흡하여, 어휘 선택과 표현이 부적절해 대화의 흐름에 부정적인 영향을 줌.
     - 50~74점: 기본적인 상황 반영은 있으나, 일부 표현이 어울리지 않거나 부적절하여 전체 분위기를 완전히 살리지 못함.
     - 75~89점: 대화의 상황과 분위기를 대부분 잘 반영하며, 어휘와 표현이 적절하나 약간의 부자연스러움이 존재함.
     - 90~100점: 모든 발언이 상황과 분위기에 완벽히 부합하며, 어휘와 표현 사용이 매우 자연스럽고 효과적임.

4) **대화의 목적과 내용을 효과적으로 구성하는가?**  
   - **세부 기준:**  
     a. 대화의 의도와 목표를 명확하게 전달하는가?  
     b. 논점을 분명하게 요약하여 전달하는가?  
     c. 청자가 흥미를 가질만한 주제와 내용을 효과적으로 구성하는가?  
   - **평가 예시:**  
     - 0~24점: 대화의 의도나 목표, 논점 전달이 전혀 이루어지지 않고, 내용 구성에 큰 혼란과 부재가 있음.
     - 25~49점: 일부 대화의 목적이나 목표가 전달되지만, 논점 요약과 내용 구성이 매우 부족하여 청자의 이해를 돕지 못함.
     - 50~74점: 대화의 기본적인 목적과 논점은 전달되지만, 내용 구성에서 명확성이나 체계적인 정리가 부족함.
     - 75~89점: 대화의 목적과 논점이 명확하게 전달되며, 내용 구성 또한 상당히 효과적이지만, 약간의 보완이 필요한 부분이 있음.
     - 90~100점: 대화의 목적이 매우 명확하며, 논점이 효과적으로 요약되고, 청자의 흥미를 끌어내는 내용 구성이 돋보임.

5) **대화 중 예의를 지키고, 긍정적인 말투와 태도를 유지했는가?**  
   - **세부 기준:**  
     a. 대화 내용에 적절한 감정 표현(극적인 표현 포함)을 사용하는가?  
     b. 청자의 입장을 존중하며, 화자의 주장을 경청하는가?  
     c. 상황과 맥락을 충분히 고려하여 응답하는가?  
     d. 화자의 이야기를 평가하거나 판단하지 않고, 공정한 태도를 유지하는가?  
   - **평가 예시:**  
     - 0~24점: 예의가 전혀 지켜지지 않고, 부정적인 말투와 공격적인 태도로 대화에 참여함.
     - 25~49점: 예의를 갖추려는 시도가 미흡하며, 부정적인 표현이나 판단적인 언급이 자주 나타나 대화 분위기를 해침.
     - 50~74점: 대체로 예의를 지키려는 모습이 보이나, 일부 표현이나 판단에서 미묘한 문제가 발생하여 개선의 여지가 있음.
     - 75~89점: 대부분의 상황에서 예의를 잘 지키며, 긍정적인 말투와 태도를 유지하지만, 소수의 상황에서 약간의 개선이 필요한 부분이 있음.
     - 90~100점: 모든 상황에서 예의 바른 태도와 긍정적인 말투를 유지하며, 상대방의 의견을 완벽하게 존중하고 공정하게 대응함.
""")

# 상대방 응답 생성 체인
opponent_response_template = PromptTemplate(
    input_variables=["conversation_history", "u_name", "op_name"],
    template=dedent("""\
        당신은 시나리오 속 인물({op_name})이다.
        시나리오상황에 집중하고, 성격을 잘 살려야한다. 
        이 상황에서 당신({op_name})이 답변한다면?

        지금까지의 대화:
        {conversation_history}

        출력 형식:
        {op_name}: "~~~"
    """)
)
opponent_chain = opponent_response_template | gpt4_llm | StrOutputParser()

# 내 답변 제안 체인
my_answer_suggest_template = PromptTemplate(
    input_variables=["conversation_history", "u_name", "op_name", "checklist"],
    template=dedent("""\
        지금까지의 대화:
        {conversation_history}

        체크리스트:
        {checklist}

        이 상황에서 상대({op_name})의 마지막 말에 대한 
        당신({u_name})의 답변을 체크리스트를 참조하고 
        80점 답변을 만들어주세요.
    """)
)
my_answer_suggest_chain = my_answer_suggest_template | gpt4_llm | StrOutputParser()

# 첨삭 체인
refine_template = PromptTemplate(
    input_variables=["conversation_history", "user_answer", "checklist", "u_name"],
    template=dedent("""\
        당신은 전문 대화 평가 전문가입니다. 제공된 대화 기록을 읽고, {u_name}의 마지막 답변을 평가해주세요.
        아래의 체크리스트와 세부 채점 루브릭에 따라 각 항목을 100점 만점으로 평가해 주세요. 각 항목에 대해 평가 점수와 그에 따른 상세한 평가 설명(근거)을 작성하고, 모든 항목의 점수를 바탕으로 전체 평가 점수(평균 혹은 가중평균)와 종합 평가 의견을 포함한 JSON 형식의 결과를 출력해 주세요.
        {checklist}
        
        [출력 형식 (JSON)]
        아래의 JSON 구조를 그대로 사용하여 평가 결과를 출력해 주세요.

        {{
        "scores": {{
            "1": {{ "score": <0~100 점>, "explanation": "<항목 1에 대한 상세 평가 설명>" }},
            "2": {{ "score": <0~100 점>, "explanation": "<항목 2에 대한 상세 평가 설명>" }},
            "3": {{ "score": <0~100 점>, "explanation": "<항목 3에 대한 상세 평가 설명>" }},
            "4": {{ "score": <0~100 점>, "explanation": "<항목 4에 대한 상세 평가 설명>" }},
            "5": {{ "score": <0~100 점>, "explanation": "<항목 5에 대한 상세 평가 설명>" }}
        }},
        "overall_score": <전체 평가 점수 (0~100)>,
        "overall_comment": "<종합 평가 의견>"
        }}
                    
        [대화 기록]
        ------------------------------------------------  
        {conversation_history}
        ------------------------------------------------  

        위의 지침과 세부 평가 기준에 따라, 제공된 대화 기록을 읽고 {u_name}의 마지막 답변을 평가해주세요. 
        각 항목에 대해 구체적인 점수와 평가 근거를 제공하고, 전체 평가 점수와 종합 의견을 포함한 JSON 결과를 출력해 주십시오.
    """)
)

#########################
# 스트리밍 평가 체인 설정
#########################
# diff=True 옵션을 사용하면 스트리밍 중 부분 JSON 응답을 받을 수 있습니다.
streaming_refinement_prompt = PromptTemplate(
    input_variables=["conversation_history", "user_answer", "checklist", "u_name"],
    template=dedent("""\
        당신은 전문 대화 평가 전문가입니다. 제공된 대화 기록을 읽고, {u_name}의 마지막 답변을 평가해주세요.
        아래의 체크리스트와 세부 채점 루브릭에 따라 각 항목을 100점 만점으로 평가해 주세요. 각 항목에 대해 평가 점수와 그에 따른 상세한 평가 설명(근거)을 작성하고, 모든 항목의 점수를 바탕으로 전체 평가 점수(평균 혹은 가중평균)와 종합 평가 의견을 포함한 JSON 형식의 결과를 출력해 주세요.
        {checklist}
        
        [출력 형식 (JSON)]
        아래의 JSON 구조를 그대로 사용하여 평가 결과를 출력해 주세요.
        
        {{
            "scores": {{
                "1": {{ "score": <0~100 점>}},
                "2": {{ "score": <0~100 점>}},
                "3": {{ "score": <0~100 점>}},
                "4": {{ "score": <0~100 점>}},
                "5": {{ "score": <0~100 점>}}
            }},
            "overall_comment": {{
                "1": {{ "explanation": "<항목 1에 대한 상세 평가 설명>"}},
                "2": {{ "explanation": "<항목 2에 대한 상세 평가 설명>"}},
                "3": {{ "explanation": "<항목 3에 대한 상세 평가 설명>"}},
                "4": {{ "explanation": "<항목 4에 대한 상세 평가 설명>"}},
                "5": {{ "explanation": "<항목 5에 대한 상세 평가 설명>"}}
            }},
            "overall_score": <전체 평가 점수 (0~100)>,
            "overall_comment": "<종합 평가 의견>"
        }}
                    
        [대화 기록]
        ------------------------------------------------  
        {conversation_history}
        ------------------------------------------------  
        
        위의 지침과 세부 평가 기준에 따라, 제공된 대화 기록을 읽고 {u_name}의 마지막 답변을 평가해주세요. 
        각 항목에 대해 구체적인 점수와 평가 근거를 제공하고, 전체 평가 점수와 종합 의견을 포함한 JSON 결과를 출력해 주십시오.
    """)
)

streaming_refinement_chain = LLMChain(
    prompt=streaming_refinement_prompt,
    llm=gpt4_llm_check,
    output_parser=JsonOutputParser(diff=True)
)

# 스트리밍 평가 체인의 응답을 비동기적으로 처리하는 함수
async def stream_refinement_result_async(data: Dict) -> Dict:
    final_result = {}
    async for partial in streaming_refinement_chain.astream(data):
        if "overall_score" in partial:
            print("\n[실시간 평가 응답]", partial)
        final_result.update(partial)
    return final_result

# 이벤트 루프가 이미 실행 중인 경우, await를 사용하여 비동기 함수 호출
# 예: Jupyter Notebook에서는 아래와 같이 실행할 수 있습니다.
refine_result = await stream_refinement_result_async({
    "conversation_history": conversation_history,
    "user_answer": user_input,
    "checklist": checklist,
    "u_name": u_name
})

#########################
# 추가 체인 설정
#########################
MOOD_THRESHOLD = 3  # 종료 임계값 (조절 가능)
mood_score = 0      # 누적 기분 지수

#########################
# 대화 데이터 기록용 리스트
#########################
conversation_records = []  # 최종 JSON에 담을 대화 기록

#########################
# 메인 루프
#########################
print("\n[대화를 시작합니다. 'quit'을 입력하면 종료합니다.]\n")
print(conversation_history)

u_name = profiles["u_name"]
op_name = profiles["op_name"]

# 초기 힌트 생성
suggested_answer = my_answer_suggest_chain.invoke({
    "conversation_history": conversation_history,
    "u_name": u_name,
    "op_name": op_name,
    "checklist": checklist
})

print(f"\n[대화 힌트]\n{suggested_answer}")

while True:
    # 사용자 입력
    user_input = input(f"\n{u_name}의 답변 (종료시 'quit'): ").strip()
    if user_input.lower() == "quit":
        print("\n--- 대화를 종료합니다. 감사합니다. ---")
        break

    if user_input.lower() == 'quit':
        print("\n--- 대화를 종료합니다. 감사합니다. ---")
        break
    
    print(f"\n[{u_name}의 답변]")
    print(user_input)

    # 대화 기록 업데이트
    conversation_history += f"\n{u_name}: {user_input}"

    # [변경] 첨삭 체인을 동기 호출하는 대신, 스트리밍 평가 체인을 사용하여 실시간 응답을 받고 최종 결과를 누적함
    refine_result = stream_refinement_result({
        "conversation_history": conversation_history,
        "user_answer": user_input,
        "checklist": checklist,
        "u_name": u_name
    })

    print(f"\n[체크리스트 평가]\n{refine_result}")

    # 상대방 응답 생성
    opponent_answer = opponent_chain.invoke({
        "conversation_history": conversation_history,
        "u_name": u_name,
        "op_name": op_name
    })
    print(f"\n[상대방 답변]\n{opponent_answer}")

    # 감정 분석 및 누적
    if refine_result['overall_score'] < 25 :
        sentiment = 2
        mood_score += sentiment
        print(f"\n[상대 기분 지수] +{sentiment} (현재 누적: {mood_score}/{MOOD_THRESHOLD})")
    elif refine_result['overall_score'] >= 25 and refine_result['overall_score'] <= 50 :
        sentiment = 1
        mood_score += sentiment
        print(f"\n[상대 기분 지수] +{sentiment} (현재 누적: {mood_score}/{MOOD_THRESHOLD})")

    if mood_score >= MOOD_THRESHOLD:
        print(f"\n⚠️ 상대방의 기분이 나빠졌습니다. (누적 지수 {mood_score})")
        print("대화가 강제 종료됩니다.")
        # 이번 턴까지 기록하고 종료
        conversation_records.append({
            "speaker": u_name,
            "text": user_input,
            "evaluation": refine_result
        })
        conversation_records.append({
            "speaker": op_name,
            "text": opponent_answer.strip(),
            "sentiment_score": sentiment
        })
        break

    # 대화 기록에 각 발화를 JSON 형태로 추가
    conversation_records.append({
        "speaker": u_name,
        "speaker_type": "user",
        "text": user_input,
        "evaluation": refine_result
    })
    conversation_records.append({
        "speaker": op_name,
        "speaker_type": "opponent",
        "text": opponent_answer.strip(),
        "sentiment_score": sentiment
    })

    # 대화 기록 업데이트
    conversation_history += f"\n{op_name}: {opponent_answer}"

    # 새로운 힌트 생성
    suggested_answer = my_answer_suggest_chain.invoke({
        "conversation_history": conversation_history,
        "u_name": u_name,
        "op_name": op_name,
        "checklist": checklist
    })
    print(f"\n[새로운 대화 힌트]\n{suggested_answer}")

print("\n--- 최종 대화 기록(JSON) ---")
# 시나리오 + 성격 + 전체 대화 기록 JSON 구조화
final_data = {
    "scenario": scenario,
    "character_personality": character_personality_1,
    "conversation_history": conversation_records
}

# JSON으로 출력
print(json.dumps(final_data, ensure_ascii=False, indent=4))


[대화를 시작합니다. 'quit'을 입력하면 종료합니다.]

당신(하민)은 20대 대학생이며,
상대(민희)을 같은 수업에서 가끔 봤지만
오늘 처음으로 학과 OT 술자리에서 제대로 대화를 나누게 되었습니다.
상대(민희)는 술잔을 들고 당신(하민)에게 말을 걸어옵니다.
사실 상대(민희)는 당신(하민)에게 관심은 없지만 
다들 친해지려고 노력하는 분위기에 단순히 맞추는 것 뿐입니다.
전혀 당신(하민)에게 관심이 없는 상태입니다.


상대(민희)의 정보:
- 21세,여자 한국인입니다.
- 솔직하고 직설적입니다.
- 시니컬한 성격입니다.
- INTP 성격 유형입니다.
- 말이 안통하면 짜증내는 성격입니다.
- 관심이 없는 상대에게는 무관심한 태도와 짧은 대답만 합니다.
- 부연설명을 하지 않습니다.

당신(하민)의 정보:
- 21세, 남자 한국인입니다.
- 소심하지만, 대화를 통해 친해지고 싶어합니다.
- 노력하는 성격입니다.


[시나리오 시작]
민희: "안녕하세요. 오늘 처음으로 제대로 대화를 나누네요. 하민씨 맞죠?"


[대화 힌트]
하민: "안녕하세요, 민희씨! 맞아요, 오늘 처음 만났네요. OT 자리에서 이렇게 모두 모여서 이야기하는 건 좋은 것 같아요. 학과 생활은 어때요? 기대되는 부분이나 걱정되는 부분이 있나요?" 

이 답변은 다음과 같은 요소를 포함하여 80점을 달성합니다:

1. **감정이나 의도 파악**:
   - 하민은 민희의 인사에 친근감을 표하며 대화에 참여하고자 하는 의도를 보입니다.

2. **상호작용**:
   - 민희에게 질문을 던져서 대화를 이어가려는 노력을 보여줍니다. 이는 민희의 반응을 고려한 상호작용입니다.

3. **상황과 분위기**:
   - OT 자리에 맞는 분위기로, 대화를 자연스럽게 이어갈 수 있는 질문을 던집니다.

4. **대화의 목적과 내용 구성**:
   - 대화의 목적은 서로를 알아가는 것이며, 학과 생활에 대한 질문을 통해 흥미로운 주제를 제시하고 있습니다.

5. **예의와 긍정적인 태도**:
   - 

RuntimeError: This event loop is already running

In [18]:
import nest_asyncio
nest_asyncio.apply()

import json
import asyncio
from textwrap import dedent
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from typing import Dict
from langchain_core.output_parsers.json import JsonOutputParser
from pydantic import BaseModel  # Pydantic 2 사용

load_dotenv(dotenv_path=".env")

#########################
# 환경설정
#########################
gpt4_llm = ChatOpenAI(
    temperature=0.7,
    model_name="gpt-4o-mini",
    max_tokens=512
)

gpt4_llm_check = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini",
    max_tokens=512
)

#########################
# 시나리오 세팅
#########################
profiles = {
    "u_name": "하민",
    "u_gender": "남자",
    "u_age": "21",
    "op_name": "민희",
    "op_gender": "여자",
    "op_age": "21"
}

scenario = dedent("""\
    당신({u_name})은 20대 대학생이며,
    상대({op_name})을 같은 수업에서 가끔 봤지만
    오늘 처음으로 학과 OT 술자리에서 제대로 대화를 나누게 되었습니다.
    상대({op_name})는 술잔을 들고 당신({u_name})에게 말을 걸어옵니다.
    사실 상대({op_name})는 당신({u_name})에게 관심은 없지만 
    다들 친해지려고 노력하는 분위기에 단순히 맞추는 것 뿐입니다.
    전혀 당신({u_name})에게 관심이 없는 상태입니다.
""").format(**profiles)

character_personality = dedent("""\
    상대({op_name})의 정보:
    - {op_age}세, {op_gender} 한국인입니다.
    - 솔직하고 직설적입니다.
    - 시니컬한 성격입니다.
    - INTP 성격 유형입니다.
    - 말이 안통하면 짜증내는 성격입니다.
    - 관심이 없는 상대에게는 무관심한 태도와 짧은 대답만 합니다.
    - 부연설명을 하지 않습니다.

    당신({u_name})의 정보:
    - {u_age}세, {u_gender} 한국인입니다.
    - 소심하지만, 대화를 통해 친해지고 싶어합니다.
    - 노력하는 성격입니다.
""").format(**profiles)

character_personality_1 = dedent("""\
    상대({op_name})의 정보:
    - {op_age}세, {op_gender} 한국인입니다.
    - 온화하고 상냥한 편입니다.
    - 타인의 감정을 잘 살피고 배려합니다.
    - ISFJ 성격 유형입니다.
    - 갈등을 피하려 하고 조용히 의견을 전달합니다.
    - 주변을 돌보는 것에 만족을 느끼는 편입니다.
    - 자기 이야기는 길게 하지 않습니다.
                                 
    당신({u_name})의 정보:
    - {u_age}세, {u_gender} 한국인입니다.
    - 소심하지만, 대화를 통해 친해지고 싶어합니다.
    - 노력하는 성격입니다.
""").format(**profiles)

init_scenario = dedent("""\
    [시나리오 시작]
    {op_name}: "안녕하세요. 오늘 처음으로 제대로 대화를 나누네요. {u_name}씨 맞죠?"
""").format(**profiles)

conversation_history = f"{scenario}\n\n{character_personality}\n\n{init_scenario}"

checklist = dedent("""\
【채점 진행 전 주의사항 및 평가 기준】  
1. 일관된 평가 기준 사용 (예: 0~24: 매우 부족, 25~49: 부족, 50~74: 보통, 75~89: 우수, 90~100: 매우 우수)
2. 평가 항목:
   1) 사용자의 감정 파악 여부
   2) 청자와의 상호작용
   3) 상황과 분위기에 맞는 대화
   4) 대화 목적 및 내용 구성
   5) 예의 및 긍정적 태도
""")

#########################
# 체인 구성 (LCEL 방식)
#########################

opponent_response_template = PromptTemplate(
    input_variables=["conversation_history", "u_name", "op_name"],
    template=dedent("""\
        당신은 시나리오 속 인물({op_name})이다.
        시나리오 상황에 집중하고, 성격을 잘 살려야 한다.
        지금까지의 대화:
        {conversation_history}

        출력 형식:
        {op_name}: "~~~"
    """)
)
opponent_chain = opponent_response_template | gpt4_llm | StrOutputParser()

my_answer_suggest_template = PromptTemplate(
    input_variables=["conversation_history", "u_name", "op_name", "checklist"],
    template=dedent("""\
        지금까지의 대화:
        {conversation_history}

        체크리스트:
        {checklist}

        이 상황에서 상대({op_name})의 마지막 말에 대한 
        당신({u_name})의 답변을 체크리스트를 참조하여 80점 수준으로 만들어주세요.
    """)
)
my_answer_suggest_chain = my_answer_suggest_template | gpt4_llm | StrOutputParser()

streaming_refinement_prompt = PromptTemplate(
    input_variables=["conversation_history", "user_answer", "checklist", "u_name"],
    template=dedent("""\
        당신은 전문 대화 평가 전문가입니다. 제공된 대화 기록을 읽고, {u_name}의 마지막 답변을 평가해주세요.
        아래 체크리스트와 평가 기준에 따라 각 항목을 100점 만점으로 평가하고, 전체 평가 점수와 종합 의견을 JSON 형식으로 출력합니다.
        
        [출력 형식 (JSON)]
        {{
            "scores": {{
                "1": {{ "score": <0~100> }},
                "2": {{ "score": <0~100> }},
                "3": {{ "score": <0~100> }},
                "4": {{ "score": <0~100> }},
                "5": {{ "score": <0~100> }}
            }},
            "overall_score": <0~100>,
            "overall_comment": "<종합 평가 의견>"
        }}
                    
        [대화 기록]
        ------------------------------------------------  
        {conversation_history}
        ------------------------------------------------  
        
        위 지침에 따라 {u_name}의 마지막 답변을 평가해주세요.
    """)
)
streaming_refinement_chain = streaming_refinement_prompt | gpt4_llm_check | JsonOutputParser(diff=True)

async def stream_refinement_result_async(data: Dict) -> Dict:
    final_result = {}
    async for partial in streaming_refinement_chain.astream(data):
        # 출력되는 partial이 dict가 아닐 경우, 안전하게 처리
        try:
            if isinstance(partial, dict):
                final_result.update(partial)
            elif isinstance(partial, (list, tuple)):
                for item in partial:
                    if isinstance(item, (list, tuple)) and len(item) == 2:
                        final_result[item[0]] = item[1]
                    else:
                        print("Ignoring non-pair item:", item)
            else:
                print("Unexpected partial type:", type(partial))
        except Exception as e:
            print("Error updating final_result with partial:", partial, e)
        print("\n[실시간 평가 응답]", partial)
    return final_result

#########################
# 메인 루프 (비동기)
#########################
async def main():
    global conversation_history  # 함수 시작 시 전역 변수 선언
    u_name = profiles["u_name"]
    op_name = profiles["op_name"]
    
    suggested_answer = my_answer_suggest_chain.invoke({
        "conversation_history": conversation_history,
        "u_name": u_name,
        "op_name": op_name,
        "checklist": checklist
    })
    print(f"\n[대화 힌트]\n{suggested_answer}")
    
    conversation_records = []
    MOOD_THRESHOLD = 3
    mood_score = 0
    
    print("\n[대화를 시작합니다. 'quit' 입력 시 종료]\n")
    print(conversation_history)
    
    while True:
        user_input = input(f"\n{u_name}의 답변 (종료시 'quit'): ").strip()
        if user_input.lower() == "quit":
            print("\n--- 대화를 종료합니다. 감사합니다. ---")
            break

        print(f"\n[{u_name}의 답변]\n{user_input}")
        updated_history = conversation_history + f"\n{u_name}: {user_input}"
    
        refine_result = await stream_refinement_result_async({
            "conversation_history": updated_history,
            "user_answer": user_input,
            "checklist": checklist,
            "u_name": u_name
        })
        print(f"\n[체크리스트 평가]\n{refine_result}")
    
        opponent_answer = opponent_chain.invoke({
            "conversation_history": updated_history,
            "u_name": u_name,
            "op_name": op_name
        })
        print(f"\n[상대방 답변]\n{opponent_answer}")
    
        overall_score = refine_result.get("overall_score", 100)
        if overall_score < 25:
            sentiment = 2
        elif 25 <= overall_score <= 50:
            sentiment = 1
        else:
            sentiment = 0
        if sentiment:
            mood_score += sentiment
            print(f"\n[상대 기분 지수] +{sentiment} (현재 누적: {mood_score}/{MOOD_THRESHOLD})")
    
        conversation_records.append({
            "speaker": u_name,
            "speaker_type": "user",
            "text": user_input,
            "evaluation": refine_result
        })
        conversation_records.append({
            "speaker": op_name,
            "speaker_type": "opponent",
            "text": opponent_answer.strip(),
            "sentiment_score": sentiment
        })
    
        conversation_history = updated_history + f"\n{op_name}: {opponent_answer}"
    
        suggested_answer = my_answer_suggest_chain.invoke({
            "conversation_history": conversation_history,
            "u_name": u_name,
            "op_name": op_name,
            "checklist": checklist
        })
        print(f"\n[새로운 대화 힌트]\n{suggested_answer}")
    
        if mood_score >= MOOD_THRESHOLD:
            print(f"\n⚠️ 상대방의 기분이 나빠졌습니다. (누적 지수 {mood_score})")
            break

    print("\n--- 최종 대화 기록(JSON) ---")
    final_data = {
        "scenario": scenario,
        "character_personality": character_personality_1,
        "conversation_history": conversation_records
    }
    print(json.dumps(final_data, ensure_ascii=False, indent=4))

if __name__ == "__main__":
    asyncio.run(main())



[대화 힌트]
하민: "안녕하세요, 민희씨! 맞아요, 오늘 처음 이야기하네요. 수업에서는 가끔 보긴 했는데 이렇게 대화하니까 신기하네요. 학과 OT는 어때요? 재밌나요?"

[대화를 시작합니다. 'quit' 입력 시 종료]

당신(하민)은 20대 대학생이며,
상대(민희)을 같은 수업에서 가끔 봤지만
오늘 처음으로 학과 OT 술자리에서 제대로 대화를 나누게 되었습니다.
상대(민희)는 술잔을 들고 당신(하민)에게 말을 걸어옵니다.
사실 상대(민희)는 당신(하민)에게 관심은 없지만 
다들 친해지려고 노력하는 분위기에 단순히 맞추는 것 뿐입니다.
전혀 당신(하민)에게 관심이 없는 상태입니다.


상대(민희)의 정보:
- 21세, 여자 한국인입니다.
- 솔직하고 직설적입니다.
- 시니컬한 성격입니다.
- INTP 성격 유형입니다.
- 말이 안통하면 짜증내는 성격입니다.
- 관심이 없는 상대에게는 무관심한 태도와 짧은 대답만 합니다.
- 부연설명을 하지 않습니다.

당신(하민)의 정보:
- 21세, 남자 한국인입니다.
- 소심하지만, 대화를 통해 친해지고 싶어합니다.
- 노력하는 성격입니다.


[시나리오 시작]
민희: "안녕하세요. 오늘 처음으로 제대로 대화를 나누네요. 하민씨 맞죠?"


[하민의 답변]
그래
Ignoring non-pair item: {'op': 'replace', 'path': '', 'value': {}}

[실시간 평가 응답] [{'op': 'replace', 'path': '', 'value': {}}]
Ignoring non-pair item: {'op': 'add', 'path': '/scores', 'value': {}}

[실시간 평가 응답] [{'op': 'add', 'path': '/scores', 'value': {}}]
Ignoring non-pair item: {'op': 'add', 'path': '/scores/1', 'value': {}}

[실시간 평가 응답] [{'op': 'add', 'path': '/scores/